In [1]:
3==3

True

In [2]:
'''
name = 상품명
storeName = 쇼핑몰 이름
category = 상품 카테고리 (outer, top, bottom, skirt, dress, others)
gender = 성별 (men, women, unisex)
brand = 브랜드명
originalPrice = 원래 가격
salePrice = 세일 가격
thumbnailUrl = 썸네일 url
contentUrl = 상세페이지 url
isSoldOut = 판매 여부
size = 실측사이즈(chest, waist)
'''

'\nname = 상품명\nstoreName = 쇼핑몰 이름\ncategory = 상품 카테고리 (outer, top, bottom, skirt, dress, others)\ngender = 성별 (men, women, unisex)\nbrand = 브랜드명\noriginalPrice = 원래 가격\nsalePrice = 세일 가격\nthumbnailUrl = 썸네일 url\ncontentUrl = 상세페이지 url\nisSoldOut = 판매 여부\nsize = 실측사이즈(chest, waist)\n'

In [3]:
import os
import json
import time
from datetime import datetime , timedelta
import re
import pandas as pd

import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [4]:
def matching_cat(cat) :
    print('matching_cat :',cat)
    # outer, top, bottom, skirt, dress, others
    res = ''
    if cat == 'OUTWEAR' :
        res = 'outer'
    if cat == 'KNIT&CARDICAN' :
        res = 'top'
    if cat == 'TOP' :
        res = 'top'
    if cat == 'DRESS' :
        res = 'dress'
    if cat == 'SLIP ' :
        res = 'others'
    if cat == 'BOTTOM' :
        res = 'bottom'
        
    return res

In [5]:
def brand_match(brand,brandlist) :
    print('brand_match :',brand)
    if any(brandlist.iloc[:,1].str.contains(brand)) :
        brand_matching = brandlist.iloc[:,0][brandlist.iloc[:,1].str.contains(brand).tolist().index(True)]
    elif any(brandlist.iloc[:,2].str.contains(brand)) :
        brand_matching = brandlist.iloc[:,0][brandlist.iloc[:,2].str.contains(brand).tolist().index(True)]
    elif any(brandlist.iloc[:,3].str.contains(brand)) :
        brand_matching = brandlist.iloc[:,0][brandlist.iloc[:,3].str.contains(brand).tolist().index(True)]
    elif any(brandlist.iloc[:,4].str.contains(brand)) :
        brand_matching = brandlist.iloc[:,0][brandlist.iloc[:,4].str.contains(brand).tolist().index(True)]
    else :
        brand_matching = 'etc'
    
    return brand_matching

In [6]:
def get_size(text,split_string) :
    print('get_size :',text)
    text=text.replace(split_string,'').split(' ')
    text=[i for i in text if i!='']
    
    size = {}
    for i,s in enumerate(text) :
        if i%2==0 :
            size[s] = 0 
            key = s
        else :
            size[key] = s
            
    if '가슴' in size.keys() :
        size['chest'] = size.pop('가슴')
    if '허리' in size.keys() :
        size['waist'] = size.pop('허리')
            
    for i in list(size.keys()) :
        if (i!='chest') and (i!='waist') :
            del size[i]
        else :
            try :
                size[i] = str(size[i])
            except :
                del size[i]
    
    return size

In [7]:
brandlist=pd.read_csv('brandlist.csv',keep_default_na=False)

In [8]:
storeName = '탐나다'
main_url = 'https://tamnada.co.kr'

cats = {'women_OUTWEAR' : ['https://tamnada.co.kr/product/list.html?cate_no=24&page=',9],
        'women_KNIT&CARDICAN' : ['https://tamnada.co.kr/product/list.html?cate_no=26&page=',12],
        'women_TOP' : ['https://tamnada.co.kr/product/list.html?cate_no=27&page=',63],
        'women_DRESS' : ['https://tamnada.co.kr/product/list.html?cate_no=68&page=',16],
        'women_SLIP' : ['https://tamnada.co.kr/product/list.html?cate_no=368&page=',10],
        'women_BOTTOM' : ['https://tamnada.co.kr/product/list.html?cate_no=42&page=',24],
        
        'men_OUTWEAR' : ['https://tamnada.co.kr/product/list.html?cate_no=111&page=',6],
        'men_KNIT&CARDICAN' : ['https://tamnada.co.kr/product/list.html?cate_no=60&page=',5],
        'men_TOP' : ['https://tamnada.co.kr/product/list.html?cate_no=49&page=',40],
        'men_BOTTOM' : ['https://tamnada.co.kr/product/list.html?cate_no=332&page=',11] 
       }

print('len(cats) :',len(cats))

len(cats) : 10


In [ ]:
%%time

start = time.time()
for c in range(len(cats)) : 
    cat = list(cats.keys())[c]
    pages = list(cats.values())[0][1]
    pages = list(range(1,pages+1))  
    
    if not os.path.exists('/'.join([storeName,cat])) :
            os.makedirs('/'.join([storeName,cat]), exist_ok=True)   
            
    cat_start = time.time()
    n_iter = 1
    for p in pages : 
        url = 'https://tamnada.co.kr/product/list.html?cate_no=24&page=' + str(p)
        html = urlopen(url)
        soup = bs(html, "html.parser")
        urls = soup.select('#info > div.description > strong') # items
        isSoldOuts = soup.select('#itsp1 > ul > li')
        isSoldOuts = [True if isSoldOuts[i].select('.promotion')[0].find('img') is not None else False for i in range(len(urls))]
        
        n = 1
        page_start = time.time()
        for u in range(len(urls)) : 
            item = main_url + urls[u].find('a').attrs['href']
            html = urlopen(item)
            soup = bs(html, "html.parser")
            
            name = str(soup.select('.name')[0])
            name = name[(name.find('<h3 class="name">')+len('<h3 class="name">')):].replace('<br/>',' ').replace('</h3>','')
            # storeName
            category = matching_cat(cat.split('_')[1])
            gender = cat.split('_')[0]
            brand = soup.select('.cont')[0].text
            brand = re.search(r'브랜드 :(.*?)컨디션 :', brand).group(1).replace(" ","").lower()
            brand = ''.join(filter(str.isalnum, brand)) 
            brand = brand_match(brand,brandlist)
            originalPrice = soup.select('#span_product_price_text')[0].text
            originalPrice = str(''.join(re.findall('\d+', originalPrice)))
            salePrice = soup.select('#span_product_price_sale')[0].text
            salePrice = salePrice[:salePrice.find('(')]
            salePrice = str(''.join(re.findall('\d+', salePrice)))
            thumbnailUrl = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.xans-element-.xans-product.xans-product-image.imgArea > div.imginfo > div.keyImg > div > img'
            thumbnailUrl = 'https:' + soup.select(thumbnailUrl)[0].attrs['src']
            contentUrl = item
            isSoldOut = isSoldOuts[0]
            sizes = soup.select('.cont')[0].text
            sizes = re.search(r'실측길이(.*?)권장사이즈', sizes).group(1).replace("( cm ) ","")
            size = get_size(sizes,':')
            
            
            result = {
                        "name":name,
                        "storeName":storeName,
                        "category":category,
                        "gender":gender,
                        "brand":brand,
                        "originalPrice":originalPrice,
                        "salePrice":salePrice,
                        "thumbnailUrl":thumbnailUrl,
                        "contentUrl":contentUrl,
                        "isSoldOut":isSoldOut,
                        "size":size
                        }
            
            file_name='/'.join([storeName,cat,'page'+str(p)+'_'+str(n)+'.json'])
            with open(file_name, "w") as json_file:
                json.dump(result, json_file)
            
            print('\n',f'{cat} {n_iter}th  pages : {p}/{pages[-1]}')
            print(result)
            print()
            if n%50 == 0 :
                print()
                print(f'category : {cat}   page - [{p}-{n}]  n_files : {n_iter}')
                print(result)
                page_run_time = timedelta(seconds=time.time() - page_start)
                print('Average - runtime : ',page_run_time.seconds/n,' seconds')
                print('category run time : ', "{:0>8}".format(str(timedelta(seconds=time.time() - cat_start))))
                print('Total run time : ', "{:0>8}".format(str(timedelta(seconds=time.time() - start))))
                print()

            n+=1
            n_iter +=1
            
            
    print(f'------------ END : {cat}')
    print('category run time : ', "{:0>8}".format(str(timedelta(seconds=time.time() - cat_start))))



print('END !!!!!!!!!!!!!!')

matching_cat : OUTWEAR
brand_match : ferratdue
get_size :  어깨 : 39 가슴 : 49 팔길이 : 54 총길이 : 58

 women_OUTWEAR 1th  pages : 1/9
{'name': 'FERRAT DUE 자켓 여성용(L)', 'storeName': '탐나다', 'category': 'outer', 'gender': 'women', 'brand': 'etc', 'originalPrice': '120000', 'salePrice': '2400', 'thumbnailUrl': 'https://tamnada.co.kr/web/product/big/25048.jpg', 'contentUrl': 'https://tamnada.co.kr/product/detail.html?product_no=166742&cate_no=24&display_group=1', 'isSoldOut': False, 'size': {'chest': '49'}}

matching_cat : OUTWEAR
brand_match : gu
get_size :  어깨 : 57 가슴 : 55 팔길이 : 12 총길이 : 102

 women_OUTWEAR 2th  pages : 1/9
{'name': 'GU 로브 여성용(L)', 'storeName': '탐나다', 'category': 'outer', 'gender': 'women', 'brand': '게스', 'originalPrice': '120000', 'salePrice': '2400', 'thumbnailUrl': 'https://tamnada.co.kr/web/product/big/24610.jpg', 'contentUrl': 'https://tamnada.co.kr/product/detail.html?product_no=166383&cate_no=24&display_group=1', 'isSoldOut': False, 'size': {'chest': '55'}}

matching_cat : 

brand_match : jpn
get_size :  어깨 : 40 가슴 : 53 팔길이 : 42 총길이 : 81

 women_OUTWEAR 18th  pages : 1/9
{'name': 'JPN 로브 여성용(L)', 'storeName': '탐나다', 'category': 'outer', 'gender': 'women', 'brand': 'etc', 'originalPrice': '120000', 'salePrice': '2400', 'thumbnailUrl': 'https://tamnada.co.kr/web/product/big/26253.jpg', 'contentUrl': 'https://tamnada.co.kr/product/detail.html?product_no=168088&cate_no=24&display_group=1', 'isSoldOut': False, 'size': {'chest': '53'}}

matching_cat : OUTWEAR
brand_match : sisisi
get_size :  어깨 : 33 가슴 : 45 팔길이 : 43 총길이 : 54

 women_OUTWEAR 19th  pages : 1/9
{'name': 'SI - SI - SI 린넨 100%자켓 여성용(M)', 'storeName': '탐나다', 'category': 'outer', 'gender': 'women', 'brand': 'etc', 'originalPrice': '170000', 'salePrice': '3400', 'thumbnailUrl': 'https://tamnada.co.kr/web/product/big/26238.jpg', 'contentUrl': 'https://tamnada.co.kr/product/detail.html?product_no=168073&cate_no=24&display_group=1', 'isSoldOut': False, 'size': {'chest': '45'}}

matching_cat : OUTWEAR
brand